In [7]:
import polars as pl
import requests
import time
import random
import logging
from typing import Dict, Any, List, Optional
from datetime import datetime

In [8]:
# import requests

# def get_vacancies_api():
#     url = "https://api.hh.ru/vacancies"

#     params = {
#         'text': 'IT',
#         'area': 1,
#         'per_page': 100,
#         'page': 0
#     }

#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
#     }

#     try:
#         response = requests.get(url, params=params, headers=headers)
#         response.raise_for_status()

#         data = response.json()

#         results = []
#         for item in data['items']:
#             results.append({
#                 'id': item['id'],
#                 'title': item['name'],
#                 'company': item['employer']['name'],
#                 'salary_from': item['salary']['from'] if item['salary'] else None,
#                 'salary_to': item['salary']['to'] if item['salary'] else None,
#                 'salary_currency': item['salary']['currency'] if item['salary'] else None,
#                 'url': item['alternate_url']
#             })

#         return results

#     except requests.RequestException as e:
#         print(f"Ошибка при запросе: {e}")
#         return []

# vacancies = get_vacancies_api()
# vacancies


https://shakhbanov.org/kak-ispolzovat-api-hhru-dlya-parsinga-sbora-vakansiy-v-oblasti-data-science/?ysclid=mf5snc647t874403230


In [9]:
# Функция для получения вакансий
def get_vacancies(city, vacancy, page):
    url = 'https://api.hh.ru/vacancies'
    params = {
        'text': f"{vacancy}",
        'per_page': 100,
        'page': page
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    return response.json()

In [10]:
# Функция для получения навыков вакансии
def get_vacancy_skills(vacancy_id: str) -> List[str]:
    """Получает навыки для конкретной вакансии"""
    try:
        url = f"https://api.hh.ru/vacancies/{vacancy_id}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Извлекаем названия навыков
        skills = data.get('key_skills', [])
        if skills and isinstance(skills, list):
            # Если навыки представлены как список словарей
            if isinstance(skills[0], dict) and 'name' in skills[0]:
                return [skill['name'] for skill in skills]
            # Если навыки представлены как список строк
            elif isinstance(skills[0], str):
                return skills
        return []
    except Exception as e:
        logging.error(f"Ошибка при получении навыков для вакансии {vacancy_id}: {e}")
        return []

In [11]:
# Функция для получения отрасли компании
def get_industry(company_id):
    # Получение отрасли компании по ее идентификатору
    if company_id is None:
        return 'Unknown'

    url = f'https://api.hh.ru/employers/{company_id}'
    response = requests.get(url)
    if response.status_code == 404:
        return 'Unknown'
    response.raise_for_status()
    data = response.json()

    if 'industries' in data and len(data['industries']) > 0:
        return data['industries'][0].get('name')
    return 'Unknown'

In [12]:
# Функция для парсинга вакансий
# Функция для парсинга вакансий
def parse_vacancies_incremental(city_id: int = 1, city: str = "Москва") -> pl.DataFrame:
    """Версия с постепенным наращиванием DataFrame"""
    vacancies = [
        'Computer vision',
        'Data Analyst', 'Data Engineer', 'Data Science', 'Data Scientist', 'ML Engineer',
        'MLOps инженер', 'AI',
        'Product Manager', 'Python Developer', 'Web Analyst', 'Аналитик данных',
        'Бизнес-аналитик', 'Системный аналитик', 'Финансовый аналитик', 'ML',
        'Deep Learning', 'NLP', 'LLM', "Project Manager", 'Product Owner','Time series',
    ]

    df_final = None
    cities = {
        'Москва': 1,
    }
    for city, city_id in cities.items():
        for vacancy in vacancies:
            page = 0
            vacancy_data = []
            logging.info(f"Парсинг вакансии: {vacancy}")
            print(f"Парсинг вакансии: {vacancy}")

            while True:
                try:
                    data = get_vacancies(city_id, vacancy, page)
                    if page==0:
                        print(f"Число страниц по запросу {data.get('pages')}")
                    if not data.get('items'):
                        break

                    for item in data['items']:
                        salary_data = item.get('salary')
                        if salary_data is None:
                            salary_from = salary_to = salary_currency = None
                            salary_str = "з/п не указана"
                        else:
                            salary_from = salary_data.get('from')
                            salary_to = salary_data.get('to')
                            salary_currency = salary_data.get('currency')
                            salary_str = f"{salary_from or ''}-{salary_to or ''} {salary_currency or ''}"

                        skills_list = get_vacancy_skills(item['id'])
                        industry = get_industry(item['employer'].get('id'))

                        # Извлекаем данные по формату работы
                        work_format = item.get('work_format', [])
                        work_format_ids = [fmt['id'] for fmt in work_format]
                        work_format_names = [fmt['name'] for fmt in work_format]

                        record = {
                            'vacancy_id': item['id'],
                            'title': item['name'],
                            'loc': item['area']['name'],
                            'requirement': item['snippet'].get('requirement', ''),
                            'responsibility': item['snippet'].get('responsibility', ''),
                            'work_format_ids': work_format_ids,
                            'work_format_names': work_format_names,
                            'skills': skills_list,
                            'company': item['employer']['name'],
                            'industry': industry,
                            'experience': item['experience'].get('name', 'Не указан'),
                            'salary_from': salary_from,
                            'salary_to': salary_to,
                            'salary_currency': salary_currency,
                            'salary_str': salary_str,
                            'url': item['alternate_url'],
                            'published_at': item.get('published_at'),
                            'source_vacancy': vacancy
                        }

                        vacancy_data.append(record)

                    if page >= data.get('pages'):
                        print(f"Страницы {page} из {data.get('pages')} закончились")
                        break

                    page += 1

                except requests.HTTPError as e:
                    logging.error(f"Ошибка при обработке вакансии {vacancy}, страница {page}: {e}")
                    print(f"Ошибка при обработке вакансии {vacancy}, страница {page}: {e}")
                    break
            time.sleep(random.uniform(3, 6))
            # Создаем DataFrame для текущей вакансии и объединяем с основным
            if vacancy_data:
                df_current = pl.DataFrame(vacancy_data)

                if df_final is None:
                    df_final = df_current
                else:
                    df_final = pl.concat([df_final, df_current])

    # Обрабатываем финальный DataFrame
    if df_final is not None:
        df_final = df_final.with_columns([
            pl.col('published_at').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%z'),
            pl.col('salary_from').cast(pl.Int64),
            pl.col('salary_to').cast(pl.Int64),
        ]).unique(subset=['vacancy_id'])

        logging.info(f"Спаршено {len(df_final)} уникальных вакансий")
        print(f"Спаршено {len(df_final)} уникальных вакансий")
        return df_final
    else:
        return pl.DataFrame()

In [ ]:
df = parse_vacancies_incremental()
df = df.with_columns(pl.col('vacancy_id').cast(pl.Int64).alias('vacancy_id'))

Парсинг вакансии: Computer vision
Число страниц по запросу 3


In [ ]:
df_unique = df.unique(subset=["vacancy_id"], keep="first")
df

In [ ]:
df

In [ ]:
df.write_parquet("vacancy.parquet")

In [ ]:
df = pl.read_parquet("vacancy.parquet")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Сохранить на Google Drive
df.write_parquet('/content/drive/MyDrive/vacancy.parquet')

In [ ]:
# Если keywords содержит строки типа "Python, ML, SQL"
# df = df.with_columns(
#     pl.col('skills').str.split(',')  # Разделяем по запятым
# )

# Теперь можно использовать explode
keyword_counts = df.explode('skills').group_by('skills').count()

In [ ]:
df

In [ ]:
keyword_counts.sort('count',descending=True)[0:10]